# Cody Deleurme

## Research question/interests

Briefly describe your research question or interests here.

In [ ]:
import pandas as pd

df = pd.read_csv("../data/raw/Highest Holywood Grossing Movies.csv")

In [ ]:
df['Genre'] = df['Genre'].str.replace(",", ' ')
df['Genre'] = df['Genre'].str.replace("]", '')
df['Genre'] = df['Genre'].str.replace("[", ' ')
genre_counts = df.Genre.str.split(expand=True).stack().value_counts()
genre_counts

C:\Users\deleu\AppData\Local\Temp\ipykernel_11896\676155851.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Genre'] = df['Genre'].str.replace("]", '')
C:\Users\deleu\AppData\Local\Temp\ipykernel_11896\676155851.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Genre'] = df['Genre'].str.replace("[", ' ')


'Adventure'      432
'Comedy'         399
'Action'         396
'Drama'          300
'Thriller'       230
'Sci-Fi'         228
'Fantasy'        227
'Family'         227
'Romance'        142
'Animation'      140
'Crime'          136
'Mystery'         87
'Musical'         51
'Horror'          45
'Biography'       39
'Music'           33
'Sport'           28
'War'             27
'History'         22
'Western'         14
'Documentary'      2
dtype: int64